In [1]:
import re
import nltk
import numpy as np
import pandas as pd
#import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

C:\Users\Sudhakor Das\anaconda3\envs\tensor\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Sudhakor Das\anaconda3\envs\tensor\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Sudhakor Das\anaconda3\envs\tensor\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
df = pd.read_csv('Bangla-Abbusive.csv', encoding = "ISO-8859-1")

In [3]:
df.head()

,Bangla,English,Reason,Categry,Label
0,????? ???? ????? ????? ???? ??? ??? ???? ?????...,Islam cannot be a religion that has to be save...,Hate towards religion,Religious,Yes
1,????? ???????? ???????? ????????,Shamim Patwari Congratulations Bangladesh,Normal Sentence,Sports,No
2,"???????? ???????? ??????? ?????, ?????? ??? ??...","Congratulations to Bangladesh cricket team, we...",Normal Sentence,Sports,No
3,"?????? ??????????, ???????, ????????? ????? ??...","Please do not catch rickshaw pullers, day labo...",Normal Sentence,Other,No
4,??? ???? ????? ?????????? ????? ??????? ????,It was very painful to see the rickshaw pulle...,Normal Sentence,Other,No


In [4]:
df.dropna(inplace = True)

In [6]:
df.isnull().sum()

Bangla     0
English    0
Reason     0
Categry    0
Label      0
dtype: int64

In [7]:
#Encoding the Dependent feature
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])

In [8]:
df.reset_index(drop = True, inplace = True)

In [9]:
df.head()

,Bangla,English,Reason,Categry,Label
0,????? ???? ????? ????? ???? ??? ??? ???? ?????...,Islam cannot be a religion that has to be save...,Hate towards religion,Religious,1
1,????? ???????? ???????? ????????,Shamim Patwari Congratulations Bangladesh,Normal Sentence,Sports,0
2,"???????? ???????? ??????? ?????, ?????? ??? ??...","Congratulations to Bangladesh cricket team, we...",Normal Sentence,Sports,0
3,"?????? ??????????, ???????, ????????? ????? ??...","Please do not catch rickshaw pullers, day labo...",Normal Sentence,Other,0
4,??? ???? ????? ?????????? ????? ??????? ????,It was very painful to see the rickshaw pulle...,Normal Sentence,Other,0


In [10]:
#Text Preprocessing
lemma = WordNetLemmatizer()
def textpreprocess(text):
    Comment = re.sub('[^a-zA-Z]', ' ', text)
    Comment = Comment.lower()
    Comment = Comment.split()
    Comment = [lemma.lemmatize(word) for word in Comment if word not in set(stopwords.words('english'))]
    Comment = ' '.join(Comment)
    
    return Comment

In [12]:
df['English'] = df['English'].apply(textpreprocess)

In [13]:
df.head()

,Bangla,English,Reason,Categry,Label
0,????? ???? ????? ????? ???? ??? ??? ???? ?????...,islam cannot religion saved killing people,Hate towards religion,Religious,1
1,????? ???????? ???????? ????????,shamim patwari congratulation bangladesh,Normal Sentence,Sports,0
2,"???????? ???????? ??????? ?????, ?????? ??? ??...",congratulation bangladesh cricket team play ma...,Normal Sentence,Sports,0
3,"?????? ??????????, ???????, ????????? ????? ??...",please catch rickshaw puller day laborer worke...,Normal Sentence,Other,0
4,??? ???? ????? ?????????? ????? ??????? ????,painful see rickshaw puller uncle screaming,Normal Sentence,Other,0


In [14]:
voc_size = 5000
one_hot = [one_hot(words, voc_size) for words in df['English']]

In [15]:
one_hot

[[3604, 3464, 4464, 4620, 2963, 72],
 [1132, 457, 4761, 2934],
 [4761, 2934, 3080, 4786, 3849, 4463, 2082, 4786, 486, 3337],
 [4656,
  268,
  4308,
  4579,
  4532,
  3557,
  1278,
  2926,
  623,
  1342,
  320,
  1717,
  734,
  2105,
  308,
  376,
  3064,
  1023],
 [2822, 2671, 4308, 4579, 1858, 4278],
 [1638, 580, 3258, 684],
 [4286, 3272, 3076, 400, 181, 2635, 1777, 185, 2136],
 [534, 1724, 534, 1036, 2787],
 [3097, 1356, 3019, 3150, 1187, 2934],
 [4492, 4254, 4131, 3314, 1677, 4505, 4045, 58, 1372, 3846, 946],
 [1356, 3019, 1052, 3846, 4157],
 [962, 2136, 225, 2105, 1796, 1082],
 [521, 2136, 2056],
 [72, 2309, 759, 3964, 424],
 [1624, 2659, 1776, 4491, 1068, 2787, 2279, 1285, 2136, 1052],
 [4492, 185, 3320, 1473, 4815],
 [4181, 1724, 346, 190, 706],
 [2858, 3034, 2666, 621],
 [355, 4618, 4049, 437],
 [386, 4730, 2926, 1114, 2139, 3375, 931],
 [381, 3172, 4372, 2925],
 [729, 4864, 4332, 4290, 1430, 3319],
 [419, 1268, 2651, 3172, 1209, 4864, 190, 1209],
 [4465, 3693, 2190, 4716, 766, 

In [16]:
sent_length = 20
Padded_onehot = pad_sequences(one_hot, padding = 'post', maxlen =sent_length )

In [17]:
Padded_onehot

array([[3604, 3464, 4464, ...,    0,    0,    0],
       [1132,  457, 4761, ...,    0,    0,    0],
       [4761, 2934, 3080, ...,    0,    0,    0],
       ...,
       [1356, 4864,  190, ...,    0,    0,    0],
       [3019,  759, 3019, ...,    0,    0,    0],
       [2910, 4741, 4730, ...,    0,    0,    0]])

In [18]:
Embedding_vector_features = 40

model = Sequential()
model.add(Embedding(voc_size, Embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer= 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
import numpy as np

In [21]:
X = np.array(Padded_onehot)
y = df['Label']
y = np.array(y)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [23]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
4/4 [==============================] - 9s 629ms/step - loss: 0.6948 - accuracy: 0.4368 - val_loss: 0.6921 - val_accuracy: 0.5361
Epoch 2/10
4/4 [==============================] - 0s 63ms/step - loss: 0.6912 - accuracy: 0.5538 - val_loss: 0.6890 - val_accuracy: 0.5155
Epoch 3/10
4/4 [==============================] - 0s 68ms/step - loss: 0.6896 - accuracy: 0.5253 - val_loss: 0.6866 - val_accuracy: 0.5052
Epoch 4/10
4/4 [==============================] - 0s 60ms/step - loss: 0.6842 - accuracy: 0.5597 - val_loss: 0.6837 - val_accuracy: 0.5258
Epoch 5/10
4/4 [==============================] - 0s 60ms/step - loss: 0.6838 - accuracy: 0.5496 - val_loss: 0.6779 - val_accuracy: 0.5464
Epoch 6/10
4/4 [==============================] - 0s 60ms/step - loss: 0.6722 - accuracy: 0.6640 - val_loss: 0.6702 - val_accuracy: 0.7320
Epoch 7/10
4/4 [==============================] - 0s 60ms/step - loss: 0.6500 - accuracy: 0.8416 - val_loss: 0.6351 - val_accuracy: 0.6907
Epoch 8/10
4/4 [==========